In [0]:

from delta.tables import *

initial_data = [
    ("P001", "Laptop", "Electronics", 999.99),
    ("P002", "Smartphone", "Electronics", 499.99),
    ("P003", "T-Shirt", "Clothing", 29.99)
]

initial_columns = ["product_id", "name", "category", "price"]

df_initial = spark.createDataFrame(initial_data, initial_columns)
df_initial.write.format("delta").mode("overwrite").saveAsTable("default.products")

spark.sql("select * from default.products").show()

+----------+----------+-----------+------+
|product_id|      name|   category| price|
+----------+----------+-----------+------+
|      P001|    Laptop|Electronics|999.99|
|      P002|Smartphone|Electronics|499.99|
|      P003|   T-Shirt|   Clothing| 29.99|
+----------+----------+-----------+------+



In [0]:

update_data = [
    ("P001", "Laptop", "Electronics", 1099.99, 50.0),
    ("P004", "Headphones", "Electronics", 149.99, 10.0),
    ("P005", "Jeans", "Clothing", 59.99, 5.0)
]

update_columns = ["product_id", "name", "category", "price", "discount"]
df_updates = spark.createDataFrame(update_data, update_columns)


delta_table = DeltaTable.forName(spark, "default.products")
df_updates.limit(0).write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("default.products")



In [0]:

delta_table.alias("target") \
    .merge(df_updates.alias("source"), "target.product_id = source.product_id") \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

spark.sql("SELECT * FROM default.products").show()

spark.sql("DESCRIBE HISTORY default.products").show(truncate=False)




+----------+----------+-----------+-------+--------+
|product_id|      name|   category|  price|discount|
+----------+----------+-----------+-------+--------+
|      P004|Headphones|Electronics| 149.99|    10.0|
|      P001|    Laptop|Electronics|1099.99|    50.0|
|      P005|     Jeans|   Clothing|  59.99|     5.0|
|      P002|Smartphone|Electronics| 499.99|    NULL|
|      P003|   T-Shirt|   Clothing|  29.99|    NULL|
+----------+----------+-----------+-------+--------+

+-------+-------------------+--------------+------------------------------+---------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+------------------------+-----------+-----------------+-------------+---------------------------------------------------------------------------------------------------

In [0]:
spark.read.format("delta").option("versionAsOf",1).table("default.products").show()


+----------+----------+-----------+------+
|product_id|      name|   category| price|
+----------+----------+-----------+------+
|      P001|    Laptop|Electronics|999.99|
|      P002|Smartphone|Electronics|499.99|
|      P003|   T-Shirt|   Clothing| 29.99|
+----------+----------+-----------+------+



In [0]:
df_before_upsert = spark.read.format("delta").option("versionAsOf", 0).table("default.products")
df_before_upsert.show()


+----------+----------+-----------+------+
|product_id|      name|   category| price|
+----------+----------+-----------+------+
|      P001|    Laptop|Electronics|999.99|
|      P002|Smartphone|Electronics|499.99|
|      P003|   T-Shirt|   Clothing| 29.99|
+----------+----------+-----------+------+



In [0]:

df2 = spark.read.table("default.products")
df2.orderBy("product_id").show()
     




+----------+----------+-----------+------+
|product_id|      name|   category| price|
+----------+----------+-----------+------+
|      P001|    Laptop|Electronics|999.99|
|      P002|Smartphone|Electronics|499.99|
|      P003|   T-Shirt|   Clothing| 29.99|
+----------+----------+-----------+------+



In [0]:

df_before_upsert = spark.read.format("delta").option("versionAsOf", 0).table("default.products")
df_before_upsert.show()

+----------+----------+-----------+------+
|product_id|      name|   category| price|
+----------+----------+-----------+------+
|      P001|    Laptop|Electronics|999.99|
|      P002|Smartphone|Electronics|499.99|
|      P003|   T-Shirt|   Clothing| 29.99|
+----------+----------+-----------+------+

